In [11]:
!pip install duckdb ydata-profiling pycaret  xgboost lightgbm catboost

^C


In [4]:
import pandas as pd
import numpy as np
import duckdb
import plotnine as p9

import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from kmodes.kmodes import KModes

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from kmodes.kprototypes import KPrototypes
from sklearn.metrics import adjusted_rand_score


In [10]:
full_train_csv_data = pd.read_csv("../playground-series-s4e12/train.csv")
full_test_csv_data = pd.read_csv("../playground-series-s4e12/test.csv")

In [11]:
len(full_test_csv_data)

800000

In [12]:
len(full_train_csv_data)

1200000

In [13]:
import pandas as pd
from pycaret.regression import setup, compare_models, add_metric

In [14]:
# Separate numeric and categorical columns
numeric_cols = full_test_csv_data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = full_test_csv_data.select_dtypes(include=['object']).columns

In [15]:
categorical_cols

Index(['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
       'Policy Type', 'Policy Start Date', 'Customer Feedback',
       'Smoking Status', 'Exercise Frequency', 'Property Type'],
      dtype='object')

In [16]:
def treat_dataset_pandas_init(dataset):
    dataset = dataset.copy()
    dataset['Policy Start Date'] = pd.to_datetime(dataset['Policy Start Date'])
    # Get the last date in the dataset
    last_date = dataset['Policy Start Date'].max()
    
    # Add 1 day to the last date
    reference_date = last_date + pd.Timedelta(days=1)
    
    # Calculate Customer Tenure in years from the reference date
    dataset['Customer Tenure'] = (reference_date - dataset['Policy Start Date']).dt.days / 365
    dataset['Claim Frequency'] = dataset['Previous Claims'] / dataset['Insurance Duration']
    # Extract year, month, day, hour, minute, second from 'Policy Start Date'
    dataset['Policy Start Year'] = dataset['Policy Start Date'].dt.year
    dataset['Policy Start Month'] = dataset['Policy Start Date'].dt.month
    dataset['Policy Start Day'] = dataset['Policy Start Date'].dt.day
    dataset['Policy Start Hour'] = dataset['Policy Start Date'].dt.hour
    dataset['Policy Start Minute'] = dataset['Policy Start Date'].dt.minute
    dataset['Policy Start Second'] = dataset['Policy Start Date'].dt.second
    dataset['Null columns'] = dataset.isnull().sum(axis=1)

    del dataset["id"]
    del dataset["Policy Start Date"]

    if "Premium Amount" in dataset.columns:
        dataset["Premium Amount"] = np.log10(dataset["Premium Amount"] + 1)

    dataset["Annual Income log 10"] = np.log10(dataset["Annual Income"])
    dataset["Previous Claims log"] = np.log1p(dataset["Previous Claims"])

    for col in [col for col in dataset.columns if col not in ["Premium Amount","id"]]:
        dataset[f"{col}_null"] = dataset[col].isnull()
    
    for col in categorical_cols:
        if col in dataset.columns:
            dataset[col] =  dataset[col].astype("category")
    
    return dataset

In [17]:
train_data = treat_dataset_pandas_init(full_train_csv_data)
test_data = treat_dataset_pandas_init(full_test_csv_data)

In [18]:
X_train = train_data[[col for col in train_data.columns if col != "Premium Amount"]]
y_train = train_data["Premium Amount"]
X_val = test_data[[col for col in test_data.columns if col != "Premium Amount"]]

In [42]:
def get_weights(y, num_bins=10):
    # Step 1: Calculate frequencies of each class
    # Step 1: Bin the target variable
    bin_edges = np.linspace(y.min(), y.max(), num_bins + 1)
    y_binned = np.digitize(y, bins=bin_edges, right=False)
    
    # Step 2: Calculate frequencies for each bin
    bin_counts = pd.value_counts(y_binned, sort=False)
    bin_weights = {bin_idx: 1.0 / count for bin_idx, count in bin_counts.items()}
    
    # Step 3: Assign weights to each sample
    weights = np.array([bin_weights[bin_idx] for bin_idx in y_binned])
    weights /= weights.mean()

    return weights

In [30]:
get_weights(y_train).sum()

10.999999999999993

In [8]:
from sklearn.metrics import root_mean_squared_log_error

def source_range_root_mean_squared_log_error(y_true, y_pred, *args, **kwargs):
    y_true_orig_range = np.power(10, y_true) - 1
    y_pred_orig_range = np.power(10, y_pred) - 1

    return root_mean_squared_log_error(y_true_orig_range, y_pred_orig_range, *args, **kwargs)

In [ ]:
v

# Lighbm

In [106]:
from sklearn.metrics import root_mean_squared_log_error

In [35]:
def rmsle_objective(y_pred, dataset):
    y_true = dataset.get_label()
    
    y_true = np.power(10, y_true)
    y_pred = np.power(10, y_pred)
    
    # Ensure no negative predictions
    y_pred = np.maximum(y_pred, 1e-15)
    log_diff = np.log(y_pred + 1) - np.log(y_true + 1)
    grad = 2 * log_diff / (y_pred + 1)
    hess = 2 / (y_pred + 1)**2
    
    return grad, hess

def rmsle_metric(y_pred, y_true):    
    y_true = np.power(10, y_true) - 1
    y_pred = np.power(10, y_pred) - 1

    rmsle = root_mean_squared_log_error(y_true, y_pred)
    
    return 'rmsle', rmsle, False  # False because lower is better

In [108]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import KFold


In [109]:
y_train.min()

1.3222192947339193

In [32]:
import xgboost as xgb
import lightgbm as lgb


In [130]:
from models import BinnedXGBRegressionModel

In [126]:
X_train.iloc[train_idx]

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,...,rand_var_0,rand_var_1,rand_var_2,rand_var_3,rand_var_4,rand_var_5,rand_var_6,rand_var_7,rand_var_8,rand_var_9
1067346,24.0,Male,3322.0,Single,1.0,Master's,Self-Employed,25.420095,Rural,Comprehensive,...,0.294668,0.756127,0.436232,0.172232,0.800602,0.981010,0.482679,0.359591,0.497413,0.235071
256599,53.0,Male,15774.0,Single,3.0,Bachelor's,Employed,49.383968,Rural,Basic,...,0.725363,0.676296,0.641614,0.639033,0.914842,0.633942,0.697627,0.720431,0.156563,0.599474
501662,33.0,Male,80686.0,Married,NaN,PhD,NaN,20.857803,Rural,Basic,...,0.839413,0.201352,0.756643,0.531282,0.252997,0.321940,0.955863,0.316257,0.504241,0.669762
47915,53.0,Male,12862.0,Divorced,1.0,High School,Unemployed,14.027907,Rural,Comprehensive,...,0.092349,0.727332,0.166094,0.912402,0.046725,0.479449,0.869187,0.976216,0.073194,0.172922
289220,62.0,Male,17076.0,Married,2.0,PhD,NaN,33.019563,Rural,Comprehensive,...,0.305390,0.292910,0.217076,0.234200,0.458933,0.749756,0.661455,0.230509,0.451165,0.142329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836489,37.0,Male,39120.0,Single,1.0,Bachelor's,NaN,47.939608,Urban,Comprehensive,...,0.461464,0.382753,0.337269,0.718637,0.797296,0.027897,0.318733,0.592962,0.926411,0.449853
491263,22.0,Male,8078.0,Single,0.0,High School,Employed,18.720524,Urban,Comprehensive,...,0.812748,0.774635,0.670007,0.054466,0.932249,0.305337,0.138253,0.082488,0.666993,0.988497
470924,64.0,Female,2789.0,Married,4.0,Bachelor's,NaN,3.848096,Rural,Basic,...,0.199434,0.343592,0.016318,0.853095,0.886846,0.126434,0.161287,0.762407,0.279665,0.601265
491755,21.0,Female,17121.0,Divorced,3.0,PhD,Employed,22.583500,Urban,Premium,...,0.010757,0.820673,0.252177,0.514541,0.052684,0.712962,0.724077,0.464268,0.066081,0.649447


In [43]:
from sklearn.model_selection import ShuffleSplit
import numpy as np

# Define a 60-40 split
splitter = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)

for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    X_train_fold_weights = get_weights(y_train_fold, num_bins=2)
    
    # Train model
    model = xgb.XGBRegressor(enable_categorical=True, eta=0.05, colsample_bytree=0.75, num_boost_round=500)
    model.fit(X_train_fold, y_train_fold)
    
    train_score = rmsle_metric(model.predict(X_train_fold), y_train_fold)
    val_score = rmsle_metric(model.predict(X_val_fold), y_val_fold)

    print(f"Train: {train_score}, Test: {val_score}")

Starting
Train: ('rmsle', 1.0449351019406832, False), Test: ('rmsle', 1.0444262068476886, False)
Starting
Train: ('rmsle', 1.044480313316101, False), Test: ('rmsle', 1.0462181981338883, False)
Starting
Train: ('rmsle', 1.044690347879979, False), Test: ('rmsle', 1.0449640656878332, False)
Starting
Train: ('rmsle', 1.0444970994953586, False), Test: ('rmsle', 1.0462344117368445, False)
Starting
Train: ('rmsle', 1.0449919086982777, False), Test: ('rmsle', 1.0437623704644123, False)


In [133]:
importances = pd.Series(model.get_booster().get_score(importance_type='total_gain')).reset_index()
importances.columns = ["column", "gain"]
importances["rand_var"] = importances["column"].str.contains("rand_var")

importances["better_than_probe_avg"] = (importances["gain"] > importances.loc[importances["rand_var"], "gain"].mean()) & ~importances["rand_var"]
importances =importances.sort_values(by="gain")
usable_columns = list(importances[importances["better_than_probe_avg"]]["column"].values)

In [134]:
usable_columns

['Customer Feedback',
 'Claim Frequency',
 'Annual Income_null',
 'Credit Score_null',
 'Customer Tenure',
 'Health Score_null',
 'Marital Status_null',
 'Previous Claims log',
 'Annual Income log 10',
 'Policy Start Year',
 'Customer Feedback_null',
 'Previous Claims',
 'Health Score',
 'Annual Income',
 'Credit Score']

In [135]:
BinnedXGBRegressionModel?

Init signature:
BinnedXGBRegressionModel(
    n_bins=5,
    classifier_base_model=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...),
    regressor_base_model=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             c

# LightGBM Limited Features

In [151]:
X_train_fixed = X_train#.loc[:, usable_columns]

In [153]:
from sklearn.model_selection import ShuffleSplit
import numpy as np

# Define a 60-40 split
splitter = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)

for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train_fixed.iloc[train_idx], X_train_fixed.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # Train model
    model = xgb.XGBRegressor(enable_categorical=True, eta=0.05, colsample_bytree=0.25, num_boost_round=500)
    model.fit(X_train_fold, y_train_fold)
    
    train_score = rmsle_metric(model.predict(X_train_fold), y_train_fold)
    val_score = rmsle_metric(model.predict(X_val_fold), y_val_fold)

    print(f"Train: {train_score}, Test: {val_score}")

Starting
Train: ('rmsle', 1.044500297790616, False), Test: ('rmsle', 1.044440191554214, False)
Starting
Train: ('rmsle', 1.0441258163128846, False), Test: ('rmsle', 1.0463571755213514, False)
Starting
Train: ('rmsle', 1.04434477031865, False), Test: ('rmsle', 1.04502001057032, False)
Starting
Train: ('rmsle', 1.0440920621786398, False), Test: ('rmsle', 1.0462488477322944, False)
Starting
Train: ('rmsle', 1.0446338897059506, False), Test: ('rmsle', 1.0437807676106068, False)


In [154]:
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt

def rescale(series, from_min, from_max, to_min, to_max):
    return to_min + ((series - from_min) * (to_max - to_min)) / (from_max - from_min)

In [156]:
model = model.fit(X_train, y_train)

In [157]:
y_train_pred = model.predict(X_train)#.loc[:, usable_columns])
y_val_pred = model.predict(X_val)#.loc[:, usable_columns])


In [168]:
res_Data = full_test_csv_data[["id"]].copy()

In [169]:
res_Data["pred"] = np.power(10, y_val_pred) - 1

In [172]:
res_Data.columns = ["id","Premium Amount"]

In [173]:
res_Data.to_csv("results_201224_1828.csv",index=False,header=True)